In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D, Conv2D, BatchNormalization, Reshape, Lambda
from keras.applications import InceptionResNetV2
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
from keras import backend as K
from keras.engine.topology import Layer

import tensorflow as tf
import keras
from keras import utils
from keras.layers import Layer
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import pickle
from scipy.signal import savgol_filter
from sklearn import metrics
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from PIL import Image
import glob
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('../rxrx1-utils')
import rxrx.io as rio

Using TensorFlow backend.


In [6]:
#load and shuffle x/y ids
x_id, y_id = np.load("../../data/x_id_pos_control.npy"), np.load("../../data/y_id_pos_control.npy")

np.random.shuffle(x_id)
training_id, val_id = x_id[:int(0.85*len(x_id))], x_id[int(0.85*len(x_id)):]
num_training_samples = len(training_id)
num_val_samples = len(val_id)

print(num_training_samples)
print(num_val_samples)

6704
1184


In [7]:
# Parameters
params = {'dim': (512,512),
          'batch_size': 5,
          'n_classes': 30,
          'n_channels': 3,
          'shuffle': True}

id_dict = {'train': training_id,
           'val': val_id}

# Generators
training_generator = DataGenerator(id_dict['train'], **params)
validation_generator = DataGenerator(id_dict['val'], **params)

In [8]:
def create_model(input_shape, n_out):
    inp = Input(input_shape)
    pretrain_model = InceptionResNetV2(include_top=False, 
                                       weights='imagenet',
                                       input_tensor=inp)
    x = pretrain_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(n_out, activation="softmax")(x)

    for layer in pretrain_model.layers:
        layer.trainable = True
        
    return Model(inp, x)

In [9]:
keras.backend.clear_session()

model = create_model(input_shape=(512,512,3), 
                     n_out=params['n_classes'])

model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.001), 
              metrics=['categorical_accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 255, 255, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 255, 255, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 255, 255, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [10]:
# Train model on dataset
history = model.fit_generator(generator=training_generator,
                            steps_per_epoch=num_training_samples // params['batch_size'],
                            epochs=10,
                            verbose=1,
                            callbacks=None,
                            validation_data=validation_generator,
                            validation_steps=num_val_samples // params['batch_size'],
                            class_weight=None,
                            max_queue_size=20,
                            workers=6,
                            use_multiprocessing=True,
                            shuffle=True,
                            initial_epoch=0)

Epoch 1/10
 105/1340 [=>............................] - ETA: 16:28 - loss: 3.5104 - categorical_accuracy: 0.0381

Process ForkPoolWorker-6:
Process ForkPoolWorker-11:
Process ForkPoolWorker-12:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Process ForkPoolWorker-4:
  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
Traceback (most recent call last):
  Fil

  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/site-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "<ipython-input-5-e5b8da08527b>", line 27, in __getitem__
    X, y = self.__data_generation(list_IDs_temp)
  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "<ipython-input-5-e5b8da08527b>", line 46, in __data_generation
    X[i,] = rio.convert_tensor_to_rgb(np.load('../../data/train_parsed_pos_control/' + ID + '.npy'))
KeyboardInterrupt
  File "../rxrx1-utils/rxrx/io.py", line 83, in convert_tensor_to_rgb
    x = (t[:, :, i] / vmax) / \
  File "/media/

KeyboardInterrupt: 

------------------------

In [3]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):   
    
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
def F1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1-K.mean(f1)

In [5]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.n_classes), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = rio.convert_tensor_to_rgb(np.load('../../data/train_parsed_pos_control/' + ID + '.npy'))

            # Store class
            y[i,] = np.load('../../data/train_parsed_pos_control/y' + ID[1:] + '.npy')

        return X, y